In [2]:
# ✅ INSTALL FIRST (Colab)
# !pip install -q stable-baselines3[extra] gymnasium[box2d]

# ✅ IMPORTS
import gymnasium as gym
import numpy as np
import time
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecTransposeImage, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

# ✅ CREATE ENV
def make_env():
    env = gym.make("CarRacing-v3", render_mode="rgb_array")
    env = Monitor(env)
    return env

env = DummyVecEnv([make_env])
env = VecTransposeImage(env)  # Change (H, W, C) to (C, H, W)
env = VecFrameStack(env, n_stack=4)
env = VecNormalize(env, norm_obs=True, norm_reward=False)

# ✅ TRAIN SAC
model = SAC("CnnPolicy", env, verbose=1, buffer_size=50000, learning_rate=3e-4, batch_size=64)
start_time = time.time()
model.learn(total_timesteps=500_000)
train_time = time.time() - start_time

# ✅ SAVE MODEL

model.save("log\ppo_carracing_sac_500k.zip")

# ✅ EVALUATE
eval_env = DummyVecEnv([make_env])
eval_env = VecTransposeImage(eval_env)
eval_env = VecFrameStack(eval_env, n_stack=4)
eval_env = VecNormalize(eval_env, norm_obs=True, norm_reward=False, training=False)

rewards = []
start_eval_time = time.time()
episodes = 10

for ep in range(episodes):
    obs = eval_env.reset()
    done = False
    ep_reward = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, info = eval_env.step(action)
        ep_reward += reward[0]
    rewards.append(ep_reward)
    print(f"Episode {ep+1}: Reward = {ep_reward:.2f}")

eval_time = time.time() - start_eval_time

# ✅ RESULTS
print("\n✅ Evaluation Summary")
print(f"Average reward over {episodes} episodes: {np.mean(rewards):.2f} ± {np.std(rewards):.2f}")
print(f"Training time: {train_time/60:.2f} mins")
print(f"Evaluation time: {eval_time:.2f} sec")


Using cuda device


MemoryError: Unable to allocate 20.6 GiB for an array with shape (50000, 1, 12, 96, 96) and data type float32